In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, lil_matrix, coo_matrix
from scipy.linalg import expm
from scipy import linalg

demonstration and design of pulse sequences for
measuring:
(i) d-wave pairing
(ii) kinetic energy
(iii) energy density

gates:
(iv) pSWAP
(v) odd-SWAP

# Hamiltonian and parameters

In [7]:
J = 1
U = 2.5
u = U/J
Bx = 1
Delta_mu = 1
dBz = 1
Bz = 1

#basis = |up,0>, |down,0>, |0,up> , |0, down>
H_hop_1p = J*np.array([[0,0,1,0],[0,0,0,1],[1,0,0,0],[0,1,0,0]])
H_idle_1p = np.array([[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]])
H_Bx_1p = Bx*np.array([[0,1,0,0],[1,0,0,0],[0,0,0,1],[0,0,1,0]])
H_tilt_1p = Delta_mu*np.array([[1,0,0,0],[0,1,0,0],[0,0,-1,0],[0,0,0,-1]])/2
H_dBz_1p = dBz*np.array([[1,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,1]])/2
H_Bz_1p = Bz*np.array([[1,0,0,0],[0,-1,0,0],[0,0,1,0],[0,0,0,-1]])

#basis: |d,\phi>, |up, up>, |up, down>, |down, up>, |down, down>, |\phi, d>
##adding Hubbard U to every term!
H_idle_2p = U*np.array([[1,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,1]])
H_hop_2p = J*np.array([[0,0,1,-1,0,0],[0,0,0,0,0,0],[1,0,0,0,0,1],[-1,0,0,0,0,-1],[0,0,0,0,0,0],[0,0,1,-1,0,0]]) + H_idle_2p
H_Bx_2p = Bx*np.array([[0,0,0,0,0,0],[0,0,1,1,0,0],[0,1,0,0,1,0],[0,1,0,0,1,0],[0,0,1,1,0,0],[0,0,0,0,0,0]]) + H_idle_2p
H_tilt_2p = Delta_mu*np.array([[1,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,-1]]) + H_idle_2p
H_dBz_2p = dBz*np.array([[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,1,0,0,0],[0,0,0,-1,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]]) + H_idle_2p
H_Bz_2p = Bz*np.array([[0,0,0,0,0,0],[0,2,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,-2,0],[0,0,0,0,0,0]]) + H_idle_2p

#basis = |up,d>, |down,d>, |d,up> , |d, down>
H_hop_3p = J*np.array([[0,0,1,0],[0,0,0,1],[1,0,0,0],[0,1,0,0]]) + U*np.eye(4)
H_idle_3p = np.array([[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]) + U*np.eye(4)
H_Bx_3p = Bx*np.array([[0,-1,0,0],[-1,0,0,0],[0,0,0,-1],[0,0,-1,0]]) + U*np.eye(4)
H_tilt_3p = Delta_mu*np.array([[-1,0,0,0],[0,-1,0,0],[0,0,1,0],[0,0,0,1]])/2 + U*np.eye(4)
H_dBz_3p = dBz*np.array([[1,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,1]])/2 + U*np.eye(4)
H_Bz_3p = Bz*np.array([[1,0,0,0],[0,-1,0,0],[0,0,1,0],[0,0,0,-1]])+ U*np.eye(4)

controls = {"Hop":[H_hop_1p,H_hop_2p,H_hop_3p],"Idle":[H_idle_1p,H_idle_2p,H_idle_3p],"Tilt":[H_tilt_1p,H_tilt_2p,H_tilt_3p],"X-field":[H_Bx_1p,H_Bx_2p,H_Bx_3p],"Z-field":[H_Bz_1p,H_Bz_2p,H_Bz_3p],"Z-grad":[H_dBz_1p,H_dBz_2p,H_dBz_3p]}

#readout basis: |d,\phi>, |up, up>, |up, down>, |down, up>, |down, down>, |\phi, d>
#control basis: |dphi^+>, |dphi^->,|sing>, |trip>,|uu-dd>,|uu+dd>
dphi_plus = np.array([1,0,0,0,0,1])/np.sqrt(2)
dphi_minus = np.array([1,0,0,0,0,-1])/np.sqrt(2)
sing = np.array([0,0,1,-1,0,0])/np.sqrt(2)
trip = np.array([0,0,1,1,0,0])/np.sqrt(2)
bell_plus = np.array([0,1,0,0,1,0])/np.sqrt(2)
bell_minus = np.array([0,1,0,0,-1,0])/np.sqrt(2)

basis_change_matrix = np.array([dphi_plus,dphi_minus,sing,trip,bell_minus,bell_plus])



In [8]:
def make_unitary(seq):
    ##seq: an ordered list of controls and durations
    u1 = np.eye(4,dtype=np.complex128)
    u2 = np.eye(6,dtype=np.complex128)
    u3 = np.eye(4,dtype=np.complex128)
    for s in seq:
        H1,H2,H3 = controls[s[0]]
        t = s[1]
        u1 = linalg.expm(-1j*H1*t).dot(u1)
        u2 = linalg.expm(-1j*H2*t).dot(u2)
        u3 = linalg.expm(-1j*H3*t).dot(u3)
    return u1,u2,u3

def unitary(u,ph):
#i think i can delete this function now
    #fix this function!!
    if u == np.inf:
        return np.exp(-1j*ph)*np.array([[np.exp(-1j*ph),0],[0,np.exp(1j*ph)]])
    else:
        lam = np.sqrt(u**2/4+4)
        T = ph/lam
        if 0< u < np.inf:
            th = np.arctan(4/u)
        elif u==0:
            th = np.pi/2
        return np.exp(-1j*u*T/2)*np.array([[np.cos(lam*T)-1j*np.sin(lam*T)*np.cos(th),-1j*np.sin(lam*T)*np.sin(th)],[-1j*np.sin(lam*T)*np.sin(th),np.cos(lam*T)+1j*np.sin(lam*T)*np.cos(th)]])


def pulse_phases(u,phase_1p=3*np.pi/2):
    aveph12 = phase_1p*2*np.sqrt(1+u**2/16)/2
    diffph12 = np.arccos((u**2/16)/(-np.cos(aveph12)))
    phi_hop1, phi_hop2 = [aveph12 + diffph12,aveph12 - diffph12]

    vx1,vy1 = np.array([(u/(4+u**2/4))*(1-np.cos(phi_hop1)),-(2/np.sqrt(4+u**2/4))*np.sin(phi_hop1)])/np.sqrt((u/(4+u**2/4))**2*(1-np.cos(phi_hop1))**2 + (4/(4+u**2/4))*np.sin(phi_hop1)**2)

    vx2,vy2 = np.array([-(u/(4+u**2/4))*(1-np.cos(phi_hop2)),-(2/np.sqrt(4+u**2/4))*np.sin(phi_hop2)])/np.sqrt((u/(4+u**2/4))**2*(1-np.cos(phi_hop2))**2 + (4/(4+u**2/4))*np.sin(phi_hop2)**2)

    phi_idle2 = np.mod(np.arctan2(vx1*vy2 - vx2*vy1,vx1*vx2 + vy1*vy2),2*np.pi)
    
    return phi_hop1, phi_hop2, phi_idle2

# d-wave correlator

In [9]:
## operator
op_dwave = [
    np.array([[0,0,0,1],[0,0,-1,0],[0,-1,0,0],[1,0,0,0]]),
    np.array([[0,-1,0,0,-1,0],[-1,0,0,0,0,-1],[0,0,0,0,0,0],[0,0,0,0,0,0],[-1,0,0,0,0,-1],[0,-1,0,0,-1,0]]),
    np.array([[0,0,0,-1],[0,0,1,0],[0,1,0,0],[-1,0,0,0]])
]

In [11]:
phi_hop1, phi_hop2, phi_idle2 = pulse_phases(u)
phi_B = np.pi/2
t_B = (phi_B/2)/(Bx)
t_hop1 = (phi_hop1/2)/np.sqrt(U**2/4+4*J**2)
t_idle2 = (phi_idle2/2)/(U/2)
t_hop2 = (phi_hop2/2)/np.sqrt(U**2/4+4*J**2)


#finding idle time 1
seq_dwave_no_idle = [["X-field",t_B],["Hop",t_hop1],["Idle",t_idle2],["Hop",t_hop2]]
us0 = make_unitary(seq_dwave_no_idle)
phi_idle1 = np.mod(np.angle(np.vdot(sing,us0[1].dot(dphi_plus)))-np.angle(np.vdot(trip,us0[1].dot(bell_plus))),2*np.pi)
t_idle1 = (phi_idle1/2)/(U/2)

#full sequence
seq_dwave = [["X-field",t_B]
          ,["Idle",t_idle1]
          ,["Hop",t_hop1],["Idle",t_idle2],["Hop",t_hop2]]
us = make_unitary(seq_dwave)

In [13]:
op_eigsys = [np.linalg.eigh(op) for op in op_dwave]

In [14]:
for j in [0,1,2]:
    print("{}-p sector".format(j+1))
    ovals, ovecs = op_eigsys[j]
    for ii in range(len(ovals)):
        if not np.isclose(ovals[ii],0):
            print(np.round(ovals[ii],5),np.round(np.abs(us[j].dot(ovecs[:,ii]))**2,6))

1-p sector
-1.0 [0.  0.5 0.5 0. ]
-1.0 [0.  0.5 0.5 0. ]
1.0 [0.5 0.  0.  0.5]
1.0 [0.5 0.  0.  0.5]
2-p sector
-2.0 [0. 0. 1. 0. 0. 0.]
2.0 [0. 0. 0. 1. 0. 0.]
3-p sector
-1.0 [0.  0.5 0.5 0. ]
-1.0 [0.  0.5 0.5 0. ]
1.0 [0.5 0.  0.  0.5]
1.0 [0.5 0.  0.  0.5]


# kinetic energy

In [15]:
phi_hop1, phi_hop2, phi_idle2 = pulse_phases(u)
phi_LR = np.pi/2
t_LR = (phi_LR/2)/(Delta_mu/2)
#t_idle1 = (phi_idle1/2)/(U/2)
t_hop1 = (phi_hop1/2)/np.sqrt(U**2/4+4*J**2)
t_idle2 = (phi_idle2/2)/(U/2)
t_hop2 = (phi_hop2/2)/np.sqrt(U**2/4+4*J**2)

#finding idle time 1
seq_KE_no_idle = [["Tilt",t_LR],["Hop",t_hop1],["Idle",t_idle2],["Hop",t_hop2]]
us0 = make_unitary(seq_KE_no_idle)
phi_idle1_KE = -np.mod(np.angle(us0[1].dot(sing)[0])-np.angle(us0[1].dot(dphi_plus)[0]),2*np.pi)
t_idle_KE = (phi_idle1_KE/2)/(U/2)

In [16]:
seq_KE = [["Tilt",t_LR]
          ,["Idle",t_idle_KE]
          ,["Hop",t_hop1],["Idle",t_idle2],["Hop",t_hop2]]

us = make_unitary(seq_KE)

In [17]:
#kinetic_energy_operators (subtracting interaction terms)
op_eigsys = [np.linalg.eigh(op) for op in [H_hop_1p,H_hop_2p-H_idle_2p,H_hop_3p-H_idle_3p]]

In [18]:
for j in [0,1,2]:
    print("{}-p sector".format(j+1))
    ovals, ovecs = op_eigsys[j]
    for ii in range(len(ovals)):
        if not np.isclose(ovals[ii],0):
            print(np.round(ovals[ii],5),np.round(np.abs(us[j].dot(ovecs[:,ii]))**2,6))

1-p sector
-1.0 [1. 0. 0. 0.]
-1.0 [0. 1. 0. 0.]
1.0 [0. 0. 0. 1.]
1.0 [0. 0. 1. 0.]
2-p sector
-2.0 [0. 0. 0. 0. 0. 1.]
2.0 [1. 0. 0. 0. 0. 0.]
3-p sector
-1.0 [0. 0. 1. 0.]
-1.0 [0. 0. 0. 1.]
1.0 [0. 1. 0. 0.]
1.0 [1. 0. 0. 0.]


# full energy density

In [20]:
#STO pulse
phi_dBz = np.pi/2
t_dBz = (phi_dBz/2)/(dBz)

### step zero: mapping eigenstates of energy density to sing \pm d\phi^+ states

In [21]:
en_vals, en_vecs = np.linalg.eigh(H_hop_2p-3*H_idle_2p/4)
print("Energy density eigvals: {}".format(np.round(en_vals,4)))
thetap = np.arctan(16/u)
z = np.cos(thetap)
theta = np.arctan(4/u)
Delta = np.sin(theta)
##determining parameters of zeroth step
#Eq. (E8, E9)
phi_hop0 = np.arccos((Delta*np.cos(theta)-z)/(np.sqrt(1-Delta**2)*np.sin(theta)))
phi_idle0 = np.arccos((Delta*np.sin(theta)+np.sqrt(1-Delta**2)*np.cos(theta)*np.cos(phi_hop0))/np.sqrt(1-z**2))
##checking against y-equation to determine sign
print(np.round(np.sqrt(1-z**2)*np.sin(phi_idle0)/(-np.sqrt(1-Delta**2)*np.sin(phi_hop0)),3))
#pulse times
t_idle0 = (np.mod(-phi_idle0,2*np.pi)/2)/(U/2)
t_hop0 = (phi_hop0/2)/np.sqrt(U**2/4+4*J**2)

seq_pre = [["Idle",t_idle0],["Hop",t_hop0]]

us_pre = make_unitary(seq_pre)
print(np.abs(np.vdot((sing-dphi_plus)/np.sqrt(2),us_pre[1].dot(en_vecs[:,0])))**2)
print(np.abs(np.vdot((sing+dphi_plus)/np.sqrt(2),us_pre[1].dot(en_vecs[:,-1])))**2)

Energy density eigvals: [-1.7118 -0.      0.      0.      0.625   2.3368]
-1.0
0.9999999999999998
0.9999999999999993


### figuring out idling time 1

In [22]:
seq_full_energy_no_idle = [["Idle",t_idle0],["Hop",t_hop0]
                           ,["Tilt",t_LR],["Hop",t_hop1],["Idle",t_idle2],["Hop",t_hop2]
                           ,["Z-grad",t_dBz]
                           ,["Hop",t_hop1],["Idle",t_idle2],["Hop",t_hop2]
                           ,["Hop",t_hop1],["Idle",t_idle2],["Hop",t_hop2]              
                          ]

us0 = make_unitary(seq_full_energy_no_idle)
t_idle_dphi_minus = (1/u)*(np.angle(np.dot(sing,us0[1].dot(dphi_minus)))-np.angle(np.dot(trip,us0[1].dot(dphi_minus))))

### figuring out idling time 2

In [23]:
seq_full_energy_idle_dphi_minus = [["Idle",t_idle0],["Hop",t_hop0]
                           ,["Tilt",t_LR],["Hop",t_hop1],["Idle",t_idle2],["Hop",t_hop2]
                           ,["Z-grad",t_dBz]
                           ,["Hop",t_hop1],["Idle",t_idle2],["Hop",t_hop2]
                           ,["Idle",t_idle_dphi_minus]
                           ,["Hop",t_hop1],["Idle",t_idle2],["Hop",t_hop2]              
                          ]

us1 = make_unitary(seq_full_energy_idle_dphi_minus)

t_idle_KE = -(1/u)*(np.angle(np.dot(dphi_plus,us1[1].dot(en_vecs[:,-1])))-np.angle(np.dot(dphi_minus,us1[1].dot(en_vecs[:,-1]))))

seq_full_energy_with_idle = [["Idle",t_idle0],["Hop",t_hop0]
                           ,["Tilt",t_LR]
                           ,["Idle",t_idle_KE]
                           ,["Hop",t_hop1],["Idle",t_idle2],["Hop",t_hop2]
                           ,["Z-grad",t_dBz]
                           ,["Hop",t_hop1],["Idle",t_idle2],["Hop",t_hop2]
                           ,["Idle",t_idle_dphi_minus]
                           ,["Hop",t_hop1],["Idle",t_idle2],["Hop",t_hop2]              
                          ]

us = make_unitary(seq_full_energy_with_idle)

In [24]:
#kinetic_energy_operators (partially subtracting interaction terms)
op_eigsys = [np.linalg.eigh(op) for op in [H_hop_1p,H_hop_2p-(3/4)*H_idle_2p,H_hop_3p-(3/4)*H_idle_3p]]

for j in [0,1,2]:
    print("{}-p sector".format(j+1))
    ovals, ovecs = op_eigsys[j]
    for ii in range(len(ovals)):
        if not np.isclose(ovals[ii],0):
            print(np.round(ovals[ii],5),np.round(np.abs(us[j].dot(ovecs[:,ii]))**2,6))

1-p sector
-1.0 [0. 0. 1. 0.]
-1.0 [0. 0. 0. 1.]
1.0 [0. 1. 0. 0.]
1.0 [1. 0. 0. 0.]
2-p sector
-1.71177 [0. 0. 0. 0. 0. 1.]
0.625 [0. 0. 1. 0. 0. 0.]
2.33677 [1. 0. 0. 0. 0. 0.]
3-p sector
-0.375 [1. 0. 0. 0.]
-0.375 [0. 1. 0. 0.]
1.625 [0. 0. 0. 1.]
1.625 [0. 0. 1. 0.]


# pSWAP

In [75]:
phi_hop1, phi_hop2, phi_idle2 = pulse_phases(u,phase_1p=3*np.pi/2)

t_hop1 = (phi_hop1/2)/np.sqrt(U**2/4+4*J**2)
t_idle2 = (phi_idle2/2)/(U/2)
t_hop2 = (phi_hop2/2)/np.sqrt(U**2/4+4*J**2)

seq_SWAP_first_hop = [["Hop",t_hop1],["Idle",t_idle2],["Hop",t_hop2]
            ,["Hop",t_hop1],["Idle",t_idle2],["Hop",t_hop2]
           ]

us_SWAP0 = make_unitary(seq_SWAP_first_hop)

t_idle_01 = (np.angle(np.vdot(sing,us_SWAP0[1].dot(sing)))-np.pi)/u
t_idle_02 = np.angle(np.vdot(dphi_plus,us_SWAP0[1].dot(dphi_plus)))/u

seq_SWAP_first_idle = [["Hop",t_hop1],["Idle",t_idle2],["Hop",t_hop2]
            ,["Idle",t_idle_01]
            ,["Hop",t_hop1],["Idle",t_idle2],["Hop",t_hop2]
            ,["Idle",t_idle_02]
            #,["Tilt",t_LR]
            #,["Hop",-t_hop1_three],["Idle",-t_idle2_three],["Hop",-t_hop2_three]
            #,["Hop",-t_hop1_three],["Idle",-t_idle2_three],["Hop",-t_hop2_three]
                     ]

us_SWAP1 = make_unitary(seq_SWAP_first_idle)

print("1-p")
print(np.round(us_SWAP1[0].dot([1,0,0,0]),6))
print(np.round(us_SWAP1[0].dot([0,1,0,0]),6))
print(np.round(us_SWAP1[0].dot([0,0,1,0]),6))
print(np.round(us_SWAP1[0].dot([0,0,0,1]),6))

print("2-p")
print("dphi+ ->"+"{}".format(np.round(basis_change_matrix.dot(us_SWAP1[1].dot(dphi_plus)),3)))
print("dphi- ->"+"{}".format(np.round(basis_change_matrix.dot(us_SWAP1[1].dot(dphi_minus)),3)))
print("sing ->"+"{}".format(np.round(basis_change_matrix.dot(us_SWAP1[1].dot(sing)),3)))
print("trip ->"+"{}".format(np.round(basis_change_matrix.dot(us_SWAP1[1].dot(trip)),3)))

print("3-p")
print(np.round(us_SWAP1[2].dot([1,0,0,0]),6))
print(np.round(us_SWAP1[2].dot([0,1,0,0]),6))
print(np.round(us_SWAP1[2].dot([0,0,1,0]),6))
print(np.round(us_SWAP1[2].dot([0,0,0,1]),6))

1-p
[-0.+0.j  0.+0.j  0.+1.j  0.+0.j]
[ 0.+0.j -0.+0.j  0.+0.j  0.+1.j]
[ 0.+1.j  0.+0.j -0.+0.j  0.+0.j]
[ 0.+0.j  0.+1.j  0.+0.j -0.+0.j]
2-p
dphi+ ->[ 1.-0.j  0.+0.j  0.+0.j -0.+0.j  0.+0.j  0.+0.j]
dphi- ->[-0.+0.j -1.-0.j  0.+0.j  0.-0.j  0.+0.j  0.+0.j]
sing ->[ 0.-0.j -0.-0.j -1.+0.j  0.-0.j  0.+0.j  0.+0.j]
trip ->[-0.-0.j -0.+0.j -0.+0.j  1.-0.j  0.+0.j  0.+0.j]
3-p
[-0.+0.j  0.+0.j  0.-1.j  0.+0.j]
[ 0.+0.j -0.+0.j  0.+0.j  0.-1.j]
[0.-1.j 0.+0.j 0.+0.j 0.+0.j]
[0.+0.j 0.-1.j 0.+0.j 0.+0.j]


# odd-SWAP

In [76]:
phi_hop1, phi_hop2, phi_idle2 = pulse_phases(u,phase_1p=3*np.pi/2)

t_hop1 = (phi_hop1/2)/np.sqrt(U**2/4+4*J**2)
t_idle2 = (phi_idle2/2)/(U/2)
t_hop2 = (phi_hop2/2)/np.sqrt(U**2/4+4*J**2)

seq_SWAP_first_hop = [["Hop",t_hop1],["Idle",t_idle2],["Hop",t_hop2]
            ,["Hop",t_hop1],["Idle",t_idle2],["Hop",t_hop2]
           ]

us_SWAP0 = make_unitary(seq_SWAP_first_hop)

t_idle_01 = (np.angle(np.vdot(sing,us_SWAP0[1].dot(sing))))/u
t_idle_02 = np.angle(np.vdot(dphi_plus,us_SWAP0[1].dot(dphi_plus)))/u

seq_SWAP_first_idle = [["Hop",t_hop1],["Idle",t_idle2],["Hop",t_hop2]
            ,["Idle",t_idle_01]
            ,["Hop",t_hop1],["Idle",t_idle2],["Hop",t_hop2]
            ,["Idle",t_idle_02]
            #,["Tilt",t_LR]
            #,["Hop",-t_hop1_three],["Idle",-t_idle2_three],["Hop",-t_hop2_three]
            #,["Hop",-t_hop1_three],["Idle",-t_idle2_three],["Hop",-t_hop2_three]
                     ]

us_SWAP1 = make_unitary(seq_SWAP_first_idle)

print("1-p")
print(np.round(us_SWAP1[0].dot([1,0,0,0]),6))
print(np.round(us_SWAP1[0].dot([0,1,0,0]),6))
print(np.round(us_SWAP1[0].dot([0,0,1,0]),6))
print(np.round(us_SWAP1[0].dot([0,0,0,1]),6))

print("2-p")
print("dphi+ ->"+"{}".format(np.round(basis_change_matrix.dot(us_SWAP1[1].dot(dphi_plus)),3)))
print("dphi- ->"+"{}".format(np.round(basis_change_matrix.dot(us_SWAP1[1].dot(dphi_minus)),3)))
print("sing ->"+"{}".format(np.round(basis_change_matrix.dot(us_SWAP1[1].dot(sing)),3)))
print("trip ->"+"{}".format(np.round(basis_change_matrix.dot(us_SWAP1[1].dot(trip)),3)))

print("3-p")
print(np.round(us_SWAP1[2].dot([1,0,0,0]),6))
print(np.round(us_SWAP1[2].dot([0,1,0,0]),6))
print(np.round(us_SWAP1[2].dot([0,0,1,0]),6))
print(np.round(us_SWAP1[2].dot([0,0,0,1]),6))

1-p
[-0.+0.j  0.+0.j  0.+1.j  0.+0.j]
[ 0.+0.j -0.+0.j  0.+0.j  0.+1.j]
[ 0.+1.j  0.+0.j -0.+0.j  0.+0.j]
[ 0.+0.j  0.+1.j  0.+0.j -0.+0.j]
2-p
dphi+ ->[ 1.-0.j  0.-0.j -0.+0.j -0.+0.j  0.+0.j  0.+0.j]
dphi- ->[ 0.+0.j  1.+0.j -0.+0.j  0.-0.j  0.+0.j  0.+0.j]
sing ->[ 0.+0.j -0.+0.j  1.-0.j  0.+0.j  0.+0.j  0.+0.j]
trip ->[-0.-0.j  0.+0.j -0.+0.j  1.-0.j  0.+0.j  0.+0.j]
3-p
[ 0.-0.j  0.+0.j -0.+1.j  0.+0.j]
[ 0.+0.j  0.-0.j  0.+0.j -0.+1.j]
[-0.+1.j  0.+0.j -0.-0.j  0.+0.j]
[ 0.+0.j -0.+1.j  0.+0.j -0.-0.j]


# spin-resolved current operator

In [63]:
from scipy import optimize

def half_pulse_function(dph,u,aveph):
    theta= np.arctan(4/u)
    return np.cos(theta)**2+np.sin(theta)**2*np.cos(aveph+dph)+np.sin(theta)*np.sin(aveph-dph)

def pulse_phase_half(u,phase_1p=np.pi/2):
    aveph12 = phase_1p*2*np.sqrt(1+u**2/16)/2
    #diffph12 = np.arccos((u**2/16)/(-np.cos(aveph12)))
    res = optimize.root_scalar(half_pulse_function,args=(u,aveph12),x0=0,x1=np.pi)
    assert res.converged
    diffph12 = res.root
    
    phi_hop1, phi_hop2 = [aveph12 + diffph12,aveph12 - diffph12]

    vx1,vy1 = np.array([(u/(4+u**2/4))*(1-np.cos(phi_hop1)),-(2/np.sqrt(4+u**2/4))*np.sin(phi_hop1)])/np.sqrt((u/(4+u**2/4))**2*(1-np.cos(phi_hop1))**2 + (4/(4+u**2/4))*np.sin(phi_hop1)**2)
    
    theta = np.arctan(4/u)
    vx2,vy2 = np.array([np.cos(theta)*np.sin(phi_hop2),np.cos(phi_hop2)])

    phi_idle2 = np.mod(np.arctan2(vx1*vy2 - vx2*vy1,vx1*vx2 + vy1*vy2),2*np.pi)
    
    return phi_hop1, phi_hop2, phi_idle2

In [70]:
phi_hop1_half, phi_hop2_half, phi_idle2_half = pulse_phase_half(u,phase_1p=np.pi/2)

t_hop1_half = (phi_hop1_half/2)/np.sqrt(U**2/4+4*J**2)
t_idle2_half = (phi_idle2_half/2)/(U/2)
t_hop2_half = (phi_hop2_half/2)/np.sqrt(U**2/4+4*J**2)

phi_dBz = np.pi
t_dBz = (phi_dBz/2)/(dBz)

phi_LR = np.pi
t_LR = (phi_LR/2)/(Delta_mu)

seq_SWAP_0 = [["Hop",t_hop2_half],["Idle",t_idle2_half],["Hop",t_hop1_half]
                      ,["Z-grad",t_dBz]
                      ,["Tilt",t_LR]
                      #,["Hop",-t_hop2_half],["Idle",-t_idle2_half],["Hop",-t_hop1_half]
                      ]

us_SWAP0 = make_unitary(seq_SWAP_0)

t_idle_01 = (np.angle(np.vdot(dphi_plus,us_SWAP0[1].dot(np.array([1,0,0,1j,0,0])/np.sqrt(2))))-np.angle(np.vdot(sing,us_SWAP0[1].dot(np.array([1,0,0,1j,0,0])/np.sqrt(2))))-np.pi/2)/u

phi_hop1_B, phi_hop2_B, phi_idle2_B = pulse_phases(u,phase_1p=np.pi/4)

t_hop1_B = (phi_hop1_B/2)/np.sqrt(U**2/4+4*J**2)
t_idle2_B = (phi_idle2_B/2)/(U/2)
t_hop2_B = (phi_hop2_B/2)/np.sqrt(U**2/4+4*J**2)

seq_SWAP_first_idle = [["Hop",t_hop2_half],["Idle",t_idle2_half],["Hop",t_hop1_half]
                      ,["Z-grad",t_dBz]
                      ,["Tilt",t_LR]
                      ,["Idle",t_idle_01]
                      ,["Hop",t_hop2_half],["Idle",t_idle2_half],["Hop",t_hop1_half]
                      ,["Hop",t_hop1_B],["Idle",t_idle2_B],["Hop",t_hop2_B]
                      #,["Idle",t_idle_02]
                      ,["Hop",t_hop1_B],["Idle",t_idle2_B],["Hop",t_hop2_B]
                      ]

us_SWAP1 = make_unitary(seq_SWAP_first_idle)

aa=basis_change_matrix.dot(us_SWAP1[1].dot(np.array([1,0,0,1j,0,0])/np.sqrt(2)))
t_idle_02 = -(np.angle(aa[0])-np.angle(aa[1]))/u

seq_SWAP_second_idle = [["Hop",t_hop2_half],["Idle",t_idle2_half],["Hop",t_hop1_half]
                      ,["Z-grad",t_dBz]
                      ,["Tilt",t_LR]
                      ,["Idle",t_idle_01]
                      ,["Hop",t_hop2_half],["Idle",t_idle2_half],["Hop",t_hop1_half]
                      ,["Hop",t_hop1_B],["Idle",t_idle2_B],["Hop",t_hop2_B]
                      ,["Idle",t_idle_02]
                      ,["Hop",t_hop1_B],["Idle",t_idle2_B],["Hop",t_hop2_B]
                      ]

us_SWAP2 = make_unitary(seq_SWAP_second_idle)

H_sp_cur_1p = J*np.array([[0,0,0,0],[0,0,0,1j],[0,0,0,0],[0,-1j,0,0]])
H_sp_cur_2p = J*np.array([[0,0,0,1j,0,0],[0,0,0,0,0,0],[0,0,0,0,0,-1j],[1j,0,0,0,0,0],[0,0,0,0,0,0],[0,0,-1j,0,0,0]])
H_sp_cur_3p = J*np.array([[0,0,0,0],[0,0,0,1j],[0,0,0,0],[0,-1j,0,0]])

op_eigsys = [np.linalg.eigh(op) for op in [H_sp_cur_1p,H_sp_cur_2p,H_sp_cur_3p]]

for j in [0,1,2]:
    print("{}-p sector".format(j+1))
    ovals, ovecs = op_eigsys[j]
    for ii in range(len(ovals)):
        if not np.isclose(ovals[ii],0):
            print(np.round(ovals[ii],5),np.round(np.abs(us_SWAP2[j].dot(ovecs[:,ii]))**2,6))

1-p sector
-1.0 [0. 0. 0. 1.]
1.0 [0. 1. 0. 0.]
2-p sector
-1.0 [0. 0. 0. 1. 0. 0.]
-1.0 [0. 0. 0. 0. 0. 1.]
1.0 [0. 0. 1. 0. 0. 0.]
1.0 [1. 0. 0. 0. 0. 0.]
3-p sector
-1.0 [0. 0. 0. 1.]
1.0 [0. 1. 0. 0.]
